In [1]:
import os
import pathlib
import re
import math
import json
import numpy as np
import pandas as pd
import polars as pl
import joblib
import warnings

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from scipy.spatial.transform import Rotation as R
import kaggle_evaluation.cmi_inference_server

warnings.filterwarnings("ignore")

# =============================================================================
# 1) CONFIG
# =============================================================================
class Config:
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    ACC_COLS = ["acc_x", "acc_y", "acc_z"]
    ROT_COLS = ["rot_w", "rot_x", "rot_y", "rot_z"]
    BATCH_SIZE = 1

# =============================================================================
# 2) PREPROCESS & FEATURES  
# =============================================================================
def handle_quaternion_missing_values(rot_data: np.ndarray) -> np.ndarray:
    rot_cleaned = rot_data.copy()
    for i in range(len(rot_data)):
        row = rot_data[i]
        missing_count = np.isnan(row).sum()
        if missing_count == 0:
            norm = np.linalg.norm(row)
            rot_cleaned[i] = row / norm if norm > 1e-8 else [1.0, 0.0, 0.0, 0.0]
        elif missing_count == 1:
            missing_idx = np.where(np.isnan(row))[0][0]
            valid_values = row[~np.isnan(row)]
            sum_squares = np.sum(valid_values**2)
            if sum_squares <= 1.0:
                missing_value = np.sqrt(max(0, 1.0 - sum_squares))
                if i > 0 and not np.isnan(rot_cleaned[i-1, missing_idx]) and rot_cleaned[i-1, missing_idx] < 0:
                    missing_value = -missing_value
                rot_cleaned[i, missing_idx] = missing_value
            else:
                rot_cleaned[i] = [1.0, 0.0, 0.0, 0.0]
        else:
            rot_cleaned[i] = [1.0, 0.0, 0.0, 0.0]
    return rot_cleaned

def calculate_angular_velocity_from_quat(rot_data, time_delta=1/200):
    quat_values = rot_data[:, [1, 2, 3, 0]] if rot_data.shape[1] == 4 else rot_data
    num_samples = quat_values.shape[0]
    angular_vel = np.zeros((num_samples, 3))
    for i in range(num_samples - 1):
        q_t, q_t_plus_dt = quat_values[i], quat_values[i+1]
        if np.any(np.isnan(q_t)) or np.any(np.isnan(q_t_plus_dt)): 
            continue
        try:
            rot_t, rot_t_plus_dt = R.from_quat(q_t), R.from_quat(q_t_plus_dt)
            delta_rot = rot_t.inv() * rot_t_plus_dt
            angular_vel[i, :] = delta_rot.as_rotvec() / time_delta
        except ValueError:
            pass
    return angular_vel

def calculate_angular_distance(rot_data):
    quat_values = rot_data[:, [1, 2, 3, 0]] if rot_data.shape[1] == 4 else rot_data
    num_samples = quat_values.shape[0]
    angular_dist = np.zeros(num_samples)
    for i in range(num_samples - 1):
        q1, q2 = quat_values[i], quat_values[i+1]
        if np.any(np.isnan(q1)) or np.any(np.isnan(q2)): 
            continue
        try:
            r1, r2 = R.from_quat(q1), R.from_quat(q2)
            angle = np.linalg.norm((r1.inv() * r2).as_rotvec())
            angular_dist[i] = angle
        except ValueError:
            pass
    return angular_dist

def remove_gravity_from_acc(acc_data, rot_data):
    acc_values = acc_data.values if isinstance(acc_data, pd.DataFrame) else acc_data
    quat_values = rot_data.values if isinstance(rot_data, pd.DataFrame) else rot_data
    quat_scipy = quat_values[:, [1, 2, 3, 0]]
    num_samples = acc_values.shape[0]
    linear_accel = np.zeros_like(acc_values)
    gravity_world = np.array([0, 0, 9.81])
    for i in range(num_samples):
        if np.any(np.isnan(quat_scipy[i])):
            linear_accel[i, :] = acc_values[i, :]
            continue
        try:
            rotation = R.from_quat(quat_scipy[i])
            gravity_sensor_frame = rotation.apply(gravity_world, inverse=True)
            linear_accel[i, :] = acc_values[i, :] - gravity_sensor_frame
        except ValueError:
            linear_accel[i, :] = acc_values[i, :]
    return linear_accel

def add_features(df: pd.DataFrame) -> pd.DataFrame:
    # TÍNH THEO NHÓM sequence_id giống bản train
    df["acc_mag"] = np.linalg.norm(df[Config.ACC_COLS].values, axis=1)
    df["acc_mag_jerk"] = df.groupby("sequence_id")["acc_mag"].diff().fillna(0)
    df["jerk_x"], df["jerk_y"], df["jerk_z"] = np.gradient(df["acc_x"]), np.gradient(df["acc_y"]), np.gradient(df["acc_z"])
    df["jerk_magnitude"] = np.linalg.norm(df[["jerk_x", "jerk_y", "jerk_z"]].values, axis=1)

    window = 20
    for _, g in df.groupby("sequence_id"):
        df.loc[g.index, "acc_xy_corr"] = g["acc_x"].rolling(window, min_periods=1).corr(g["acc_y"]).fillna(0)
        df.loc[g.index, "acc_xz_corr"] = g["acc_x"].rolling(window, min_periods=1).corr(g["acc_z"]).fillna(0)
        df.loc[g.index, "acc_yz_corr"] = g["acc_y"].rolling(window, min_periods=1).corr(g["acc_z"]).fillna(0)

    df["rot_angle"] = 2 * np.arccos(df["rot_w"].clip(-1, 1))
    df["rot_angle_vel"] = df.groupby("sequence_id")["rot_angle"].diff().fillna(0)

    rot_numpy = df[Config.ROT_COLS].to_numpy()
    angular_vel = calculate_angular_velocity_from_quat(rot_numpy)
    angular_dist = calculate_angular_distance(rot_numpy)
    df[["angular_vel_x", "angular_vel_y", "angular_vel_z"]] = angular_vel
    df["angular_distance"] = angular_dist
    df["angular_vel_magnitude"] = np.linalg.norm(angular_vel, axis=1)

    linear_accel = remove_gravity_from_acc(df[Config.ACC_COLS], df[Config.ROT_COLS])
    df[["acc_x2", "acc_y2", "acc_z2"]] = linear_accel
    df["acc_mag2"] = np.linalg.norm(linear_accel, axis=1)
    df["acc_mag_jerk2"] = df.groupby("sequence_id")["acc_mag2"].diff().fillna(0)
    df["jerk_x2"], df["jerk_y2"], df["jerk_z2"] = np.gradient(df["acc_x2"]), np.gradient(df["acc_y2"]), np.gradient(df["acc_z2"])
    df["jerk_magnitude2"] = np.linalg.norm(df[["jerk_x2", "jerk_y2", "jerk_z2"]].values, axis=1)

    for _, g in df.groupby("sequence_id"):
        df.loc[g.index, "acc_xy_corr2"] = g["acc_x2"].rolling(window, min_periods=1).corr(g["acc_y2"]).fillna(0)
        df.loc[g.index, "acc_xz_corr2"] = g["acc_x2"].rolling(window, min_periods=1).corr(g["acc_z2"]).fillna(0)
        df.loc[g.index, "acc_yz_corr2"] = g["acc_y2"].rolling(window, min_periods=1).corr(g["acc_z2"]).fillna(0)

    df.replace([np.inf, -np.inf], 0, inplace=True)
    df.fillna(0, inplace=True)
    return df

def preprocess_left_handed_pd(df: pd.DataFrame, demo: pd.DataFrame) -> pd.DataFrame:
    pl_df = pl.DataFrame(df)
    pl_demo = pl.DataFrame(demo)
    l_subjs = pl_demo.filter(pl.col("handedness") == 0)["subject"].to_list()

    r_tr = pl_df.filter(~pl.col("subject").is_in(l_subjs))
    l_tr = pl_df.filter(pl.col("subject").is_in(l_subjs))
    if l_tr.shape[0] == 0:
        return r_tr.to_pandas()

    # Flip trục y, z gia tốc; mirror quaternion như bản train
    l_tr = l_tr.with_columns(-pl.col("acc_y"))
    l_tr = l_tr.with_columns(-pl.col("acc_z"))

    rot_np = l_tr.select(Config.ROT_COLS).to_numpy()
    rot_scipy = rot_np[:, [1, 2, 3, 0]]
    r = R.from_quat(rot_scipy)
    euler = r.as_euler("xyz")
    euler[:, 1] = -euler[:, 1]
    euler[:, 2] = -euler[:, 2]
    r = R.from_euler("xyz", euler)
    q = r.as_quat()
    l_tr = l_tr.with_columns(pl.DataFrame(q, schema=["rot_x", "rot_y", "rot_z", "rot_w"]))

    pl_df2 = pl.concat([r_tr, l_tr]).sort(by="row_id")
    return pl_df2.to_pandas()

def pad_sequences(sequences: list, maxlen: int, padding: str="pre", truncating: str="pre", dtype: str="float32") -> np.ndarray:
    n_samples = len(sequences); sample_shape = tuple()
    for s in sequences:
        if len(s) > 0:
            sample_shape = np.asarray(s).shape[1:]; break
    x = np.zeros((n_samples, maxlen) + sample_shape, dtype=dtype)
    for idx, s in enumerate(sequences):
        if len(s) == 0: continue
        s_np = np.asarray(s, dtype=dtype)
        if truncating == "pre": s_np = s_np[-maxlen:]
        else: s_np = s_np[:maxlen]
        trunc = s_np.shape[0]
        if padding == "pre": x[idx, -trunc:] = s_np
        else: x[idx, :trunc] = s_np
    return x

# --- Pyramid Temporal Feature (PTF) ---
class PyramidTemporalFeature(nn.Module):
    """
    Module trích xuất đặc trưng theo nhiều tần suất thời gian khác nhau.
    Tương tự FPN nhưng dùng nhiều Conv1D kernel size khác nhau để nắm bắt pattern ngắn & dài.
    """
    def __init__(self, in_channels, out_channels, kernel_sizes=[3, 5, 7]):
        super().__init__()
        self.branches = nn.ModuleList([
            nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=k, padding=k//2),
                nn.BatchNorm1d(out_channels),
                nn.ReLU(inplace=True)
            )
            for k in kernel_sizes
        ])
        # Gộp lại các scale
        self.fuse = nn.Conv1d(out_channels * len(kernel_sizes), out_channels, kernel_size=1)

    def forward(self, x):
        # x: [B, C, T]
        features = [branch(x) for branch in self.branches]  # nhiều scale thời gian
        concat = torch.cat(features, dim=1)  # ghép theo kênh
        out = self.fuse(concat)              # hợp nhất lại
        return out                           # [B, out_channels, T]


# =============================================================================
# 3) MODEL (giống bản train)
# =============================================================================
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=8):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction), nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel), nn.Sigmoid())
    def forward(self, x):
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y.expand_as(x)

class ResidualSEBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, drop=0.3):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding="same", bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding="same", bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.se = SEBlock(out_channels)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drop)
        self.pool = nn.MaxPool1d(2)
        self.shortcut = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, 1, bias=False),
            nn.BatchNorm1d(out_channels)
        ) if in_channels != out_channels else nn.Identity()
    def forward(self, x):
        identity = self.shortcut(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.se(self.bn2(self.conv2(out)))
        out = self.dropout(self.pool(self.relu(out + identity)))
        return out

class AttentionLayer(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attn = nn.Sequential(nn.Linear(hidden_dim, 1), nn.Tanh())

    def forward(self, x):
        weights = F.softmax(self.attn(x).squeeze(-1), dim=1).unsqueeze(-1)
        return torch.sum(x * weights, dim=1)


class CrossAttention(nn.Module):
    def __init__(self, feature_dim, num_heads=8, dropout=0.1):
        super().__init__()
        self.feature_dim = feature_dim
        self.num_heads = num_heads
        self.head_dim = feature_dim // num_heads
        assert feature_dim % num_heads == 0, "feature_dim must be divisible by num_heads"
        self.q_linear = nn.Linear(feature_dim, feature_dim, bias=False)
        self.k_linear = nn.Linear(feature_dim, feature_dim, bias=False)
        self.v_linear = nn.Linear(feature_dim, feature_dim, bias=False)
        self.out_linear = nn.Linear(feature_dim, feature_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(feature_dim)
    def forward(self, query_branch, key_value_branches):
        B, T, C = query_branch.shape
        Q = self.q_linear(query_branch)
        all_kv = torch.cat(key_value_branches, dim=1)
        K = self.k_linear(all_kv)
        V = self.v_linear(all_kv)
        Q = Q.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        K = K.view(B, -1, self.num_heads, self.head_dim).transpose(1, 2)
        V = V.view(B, -1, self.num_heads, self.head_dim).transpose(1, 2)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)
        attn_output = torch.matmul(attn_weights, V)
        attn_output = attn_output.transpose(1, 2).contiguous().view(B, T, C)
        output = self.out_linear(attn_output)
        output = self.layer_norm(query_branch + output)
        return output

class IMUCrossAttentionFusion(nn.Module):
    def __init__(self, feature_dim=256, num_heads=8, dropout=0.1):
        super().__init__()
        self.cross_attn1 = CrossAttention(feature_dim, num_heads, dropout)
        self.cross_attn2 = CrossAttention(feature_dim, num_heads, dropout)
        self.cross_attn3 = CrossAttention(feature_dim, num_heads, dropout)
    def forward(self, imu1, imu2, imu3):
        e1 = self.cross_attn1(imu1, [imu2, imu3])
        e2 = self.cross_attn2(imu2, [imu1, imu3])
        e3 = self.cross_attn3(imu3, [imu1, imu2])
        return e1, e2, e3

class IMUCrossAttentionModel_PTF(nn.Module):
    def __init__(self, imu_dim, n_classes):
        super().__init__()
        self.imu_dim = imu_dim

        # --- 3 nhánh CNN backbone cho dữ liệu IMU ---
        self.imu_branch1 = nn.Sequential(
            ResidualSEBlock(12, 128, 3),
            ResidualSEBlock(128, 256, 5)
        )
        self.imu_branch2 = nn.Sequential(
            ResidualSEBlock(11, 128, 3),
            ResidualSEBlock(128, 256, 5)
        )
        self.imu_branch3 = nn.Sequential(
            ResidualSEBlock(12, 128, 3),
            ResidualSEBlock(128, 256, 5)
        )

        # --- Pyramid Temporal Feature cho mỗi nhánh ---
        self.ptf1 = PyramidTemporalFeature(256, 128)
        self.ptf2 = PyramidTemporalFeature(256, 128)
        self.ptf3 = PyramidTemporalFeature(256, 128)

        # --- Cross attention fusion ---
        self.cross_attention_fusion = IMUCrossAttentionFusion(feature_dim=128)

        # --- BiLSTM + attention + FC ---
        self.bilstm = nn.LSTM(128 * 3, 512, bidirectional=True, batch_first=True)
        self.attention = AttentionLayer(1024)
        self.fc = nn.Linear(1024, n_classes)
    
    def forward(self, x):
        imu = x[:, :, :self.imu_dim]

        # tách dữ liệu IMU cho 3 nhánh (tuỳ theo cấu trúc cảm biến)
        imu1 = self.imu_branch1(imu[:, :, :12].transpose(1, 2))     # [B, C, T]
        imu2 = self.imu_branch2(imu[:, :, 12:23].transpose(1, 2))
        imu3 = self.imu_branch3(imu[:, :, 23:].transpose(1, 2))
        
        # --- Trích đặc trưng đa tần số thời gian ---
        imu1_ptf = self.ptf1(imu1).transpose(1, 2)
        imu2_ptf = self.ptf2(imu2).transpose(1, 2)
        imu3_ptf = self.ptf3(imu3).transpose(1, 2)

        # --- Cross attention fusion giữa 3 nhánh ---
        imu1, imu2, imu3 = self.cross_attention_fusion(imu1_ptf, imu2_ptf, imu3_ptf)
    
        # --- Ghép & đưa qua BiLSTM ---
        merged = torch.cat((imu1, imu2, imu3), dim=2)
        lstm_out, _ = self.bilstm(merged)

        # --- Attention + phân lớp ---
        attended = self.attention(lstm_out)
        return self.fc(attended)

# =============================================================================
# 4) ENS INFERENCE UTILITIES
# =============================================================================
class CMIInferenceDataset(Dataset):
    def __init__(self, data_array):
        self.data_array = data_array
    def __len__(self):
        return 1
    def __getitem__(self, idx):
        return torch.FloatTensor(self.data_array)

def _find_model_dir_and_folds():
    """
    Tìm thư mục chứa artifacts. Ưu tiên /kaggle/working, sau đó các input dir.
    Kỳ vọng tồn tại các cặp:
      - prep_fold_{k}.joblib
      - model_best_fold_{k}.pt
    Fallback: nếu không có theo fold, dùng single model 'prep.joblib' + 'model_best.pt'.
    """
    candidates = [pathlib.Path("/kaggle/working"), pathlib.Path("/kaggle/input/lstm-triet")]
    for base in candidates:
        if not base.exists():
            continue
        fold_files = sorted(base.glob("prep_fold_*.joblib"))
        model_files = sorted(base.glob("model_best_fold_*.pt"))
        if fold_files and model_files:
            # lấy list fold chung giữa 2 bên
            folds = []
            for p in fold_files:
                m = re.search(r"prep_fold_(\d+)\.joblib$", p.name)
                if m:
                    k = int(m.group(1))
                    if (base / f"model_best_fold_{k}.pt").exists():
                        folds.append(k)
            folds = sorted(set(folds))
            if folds:
                return base, folds, True
        # fallback single
        if (base / "prep.joblib").exists() and (base / "model_best.pt").exists():
            return base, [], False
    raise FileNotFoundError("Không tìm thấy artifacts theo fold hoặc single trong các thư mục dự kiến.")

def _prepare_sequence(sequence_df: pd.DataFrame, demo_df: pd.DataFrame) -> pd.DataFrame:
    # xử lý giống train
    sequence_df = sequence_df.copy()
    sequence_df[Config.ROT_COLS] = handle_quaternion_missing_values(sequence_df[Config.ROT_COLS].to_numpy())
    sequence_df = pd.concat([g.ffill().bfill().fillna(0) for _, g in sequence_df.groupby("sequence_id")], axis=0)
    sequence_df = preprocess_left_handed_pd(sequence_df, demo_df)
    sequence_df = add_features(sequence_df)
    return sequence_df

def _build_features(sequence_df: pd.DataFrame, features: list, transformer, max_length: int) -> np.ndarray:
    seq_X = sequence_df[features].to_numpy()
    # seq_X = transformer.transform(seq_X)
    if len(seq_X) > max_length:
        padded = seq_X[-max_length:]
    else:
        padded = np.zeros((max_length, seq_X.shape[1]), dtype=np.float32)
        padded[-len(seq_X):, :] = seq_X
    return padded

def create_prediction_function_ensemble(models, preps, categories):
    """
    models: list[nn.Module]
    preps : list[dict] (transformer, features, max_length, fold)
    categories: list[str]
    """
    def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
        try:
            seq_df = sequence.to_pandas()
            demo_df = demographics.to_pandas()

            required_cols = set(Config.ACC_COLS + Config.ROT_COLS + ["sequence_id", "row_id", "subject"])
            missing = required_cols - set(seq_df.columns)
            if missing:
                raise ValueError(f"Thiếu cột bắt buộc: {missing}")

            seq_df = _prepare_sequence(seq_df, demo_df)

            probs_all = []
            for model, prep in zip(models, preps):
                feats = prep["features"]
                X_pad = _build_features(seq_df, feats, prep["transformer"], prep["max_length"])
                ds = CMIInferenceDataset(X_pad)
                loader = DataLoader(ds, batch_size=Config.BATCH_SIZE)
                model.eval()
                with torch.no_grad():
                    for batch_X in loader:
                        logits = model(batch_X.to(Config.DEVICE))
                        probs = torch.softmax(logits, dim=1).cpu().numpy()
                        probs_all.append(probs[0])  # (C,)
            probs_mean = np.mean(probs_all, axis=0)
            pred_idx = int(np.argmax(probs_mean))
            return categories[pred_idx]
        except Exception as e:
            print(f"[Predict Error] {e}")
            return categories[0]
    return predict

# =============================================================================
# 5) MAIN: LOAD ENSEMBLE hoặc FALLBACK SINGLE
# =============================================================================
def main():
    model_dir, folds, is_fold = _find_model_dir_and_folds()
    print(f"Artifacts dir: {model_dir} | by_fold={is_fold} | folds={folds}")

    if is_fold:
        # kiểm tra categories giữa các fold nhất quán
        preps = []
        models = []
        cats_ref = None
        for k in folds:
            prep = joblib.load(model_dir / f"prep_fold_{k}.joblib")
            preps.append(prep)
            if cats_ref is None:
                cats_ref = prep["categories"]
            else:
                assert cats_ref == prep["categories"], "Mismatch categories giữa các fold!"
            model = IMUCrossAttentionModel_PTF(
                imu_dim=len(prep["features"]),
                n_classes=len(prep["categories"])
            ).to(Config.DEVICE)
            state = torch.load(model_dir / f"model_best_fold_{k}.pt", map_location=Config.DEVICE)
            model.load_state_dict(state)
            models.append(model)
        print(f"✓ Loaded {len(models)} fold models for ensemble.")
        predict_fn = create_prediction_function_ensemble(models, preps, cats_ref)
        return predict_fn
    else:
     
        artifacts = joblib.load(model_dir / "prep.joblib")
        features = artifacts['features']
        n_classes = len(artifacts['categories'])
        model = IMUCrossAttentionModel_PTF(imu_dim=len(features), n_classes=n_classes).to(Config.DEVICE)
        model.load_state_dict(torch.load(model_dir / "model_best.pt", map_location=Config.DEVICE))
        print("✓ Loaded single model artifacts.")

        def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
            try:
                seq_df = _prepare_sequence(sequence.to_pandas(), demographics.to_pandas())
                X_pad = _build_features(seq_df, features, artifacts["transformer"], artifacts["max_length"])
                ds = CMIInferenceDataset(X_pad)
                loader = DataLoader(ds, batch_size=Config.BATCH_SIZE)
                model.eval()
                with torch.no_grad():
                    for batch_X in loader:
                        logits = model(batch_X.to(Config.DEVICE))
                        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
                        pred_idx = int(np.argmax(probs))
                return artifacts["categories"][pred_idx]
            except Exception as e:
                print(f"[Predict Error] {e}")
                return artifacts["categories"][0]
        return predict

if __name__ == '__main__':
    predict_function = main()
    inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict_function)

    if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        inference_server.serve()
    else:
        test_csv_path = pathlib.Path("/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv")
        test_demo_path = pathlib.Path("/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv")
        if test_csv_path.exists():
            inference_server.run_local_gateway(data_paths=(test_csv_path, test_demo_path))
        else:
            print("Local test data not found.")


Artifacts dir: /kaggle/input/lstm-triet | by_fold=True | folds=[1, 2, 3, 4, 5]
✓ Loaded 5 fold models for ensemble.
